In [1]:
import pandas as pd
import sys
pd.set_option("display.max_columns", 200)


In [6]:
# Data Input

file_name = "101._Amanpur"
last_ps_no = 365
consti_name = "Etah"
total_ind_parties = 5
lst_of_parties = ["CPI","BJP","SP","INC","BSP","QED",
                  "AIPF","BMP","AAP","SSP","CPI-M","IND",
                  "IND","IND"]
# lst_of_parties = ["SP","BJP","RSSP","SBSP",	"RBP",
#                   	"BSCP",	"RKP", "PSP", "JAP",	
#                     "IND ( Ashok Kumar )",	"IND ( Indrapal )",	
#                     "IND ( Parvati Nandan )",	
#                     "IND ( Satendra Kumar Panda )", "IND ( Hari Om )"]


In [7]:
# Version 1 without NOTA Creation

# Read the file
df = pd.read_excel(r"C:\Users\Aakash D\Downloads\Analytics\Self Paced Projects\Data Cleaning Automation\101._Amanpur.xls", header = None)
#Drop the redundant rows
# for the first rows
df = df.drop(range(df[df[1] == 1].index[0])).reset_index(drop=True)

# for the last row
df = df.iloc[0:((df[df[1] == last_ps_no].index[0])+1),:]

# Setting up numerical Col Names
perma_col_alpha = 10
perma_col_omega = 2
abs_col_freq = perma_col_alpha + len(lst_of_parties) + (len(lst_of_parties)*2) + perma_col_omega

# Nota Columns Check
add_cols = (df.shape[1] - abs_col_freq)
if abs_col_freq != df.shape[1]:
  df.columns = [col for col in range(1,abs_col_freq+(add_cols + 1))]
  if add_cols == 2:
    sdw_nota = True
  elif add_cols == 1:
    sdw_nota = "one"
  else:
    sdw_nota = "Greater than 2"
    print("Multiple Columns Detected Before NOTA, Terminating Processing")
    sys.exit()
else:
  df.columns = [col for col in range(1,abs_col_freq+1)]
  sdw_nota = False

# dropping the total_electors and EPIC column
redun_cols = [4,9]
df = df.drop(columns=redun_cols, axis= 1)

df.fillna(0, inplace= True)

#--------------------------------------------------------------------------
#Segregation
# Start_part_segregation
start_part = df.iloc[:, :8]
start_part = start_part.copy()

# Seg #1
start_part_seg_1 = start_part.iloc[:, :3]
start_part_seg_1 = start_part_seg_1.copy()

start_part_seg_1["S.No."] = range(1, last_ps_no+1)
start_part_seg_1_Sno = start_part_seg_1.iloc[:, -1:].astype(int)
start_part_seg_1_Sno = start_part_seg_1_Sno.copy()

start_part_seg_1.drop(start_part_seg_1.iloc[:, -1:], axis = 1, inplace = True)
start_part_seg_1.drop(start_part_seg_1.iloc[:, [0]], axis = 1, inplace = True)

# Seg #2
start_part_seg_2 = start_part.iloc[:, 3:]
start_part_seg_2 = start_part_seg_2.copy()

seg_2_cols = [cols for cols in start_part_seg_2]
start_part_seg_2[seg_2_cols] = start_part_seg_2[seg_2_cols].replace('', 0)
start_part_seg_2[seg_2_cols] = start_part_seg_2[seg_2_cols].astype(int)

# Organization of Tendered voters
start_part_seg_2_ten = start_part_seg_2.iloc[:, -1:]

# Total Votes Calc
start_part_seg_2.drop(start_part_seg_2.columns[-2:], axis=1, inplace=True)
start_part_seg_2["Total"] = start_part_seg_2[[col for col in start_part_seg_2.columns]].sum(axis=1).astype(int)


start_part_seg_2 = pd.concat([start_part_seg_2, start_part_seg_2_ten], axis=1)


#--------------------------------------------------------------------------

# Mid Part segmentation
mid_part = df.iloc[:, 8:(abs_col_freq - perma_col_omega - len(redun_cols))] # abs party segment
mid_part = mid_part.copy()
# Cleaned party Seg
mid_part = mid_part.iloc[:, 2::3]
mid_part = mid_part.copy()

mid_part_cols = [col for col in mid_part.columns]
mid_part[mid_part_cols] = mid_part[mid_part_cols].replace('', 0)
mid_part[mid_part_cols] = mid_part[mid_part_cols].astype(int)

# Mid-part Calculation

# For IND TOTAL Calc
mid_part_ind = mid_part.iloc[:, - total_ind_parties:]
mid_part_ind = mid_part_ind.copy()
mid_part_ind["IND (TOTAL)"] = mid_part_ind[[col for col in mid_part_ind.columns]].sum(axis=1).astype(int)
mid_part_ind_seg = mid_part_ind.iloc[:, -1:]
mid_part_ind_seg = mid_part_ind_seg.copy()

# For Total Valid Votes Calc
mid_part["Total Valid Votes"] = mid_part[[col for col in mid_part.columns]].sum(axis=1).astype(int)
mid_part_val = mid_part.iloc[:, -1:]
mid_part_val = mid_part_val.copy()

mid_part.drop(mid_part.iloc[:, -1:], axis = 1, inplace=True)

#---------------------------------------------------------------------------

# End Part Segmentation
if sdw_nota == True:
  end_part = df.iloc[:, -4:]
  end_part.drop(end_part.iloc[:, 0:2], axis=1, inplace=True)

elif sdw_nota == "one":
  end_part = df.iloc[:, -3:]
  end_part.drop(end_part.iloc[:, [0]], axis=1, inplace=True)

else:
  sdw_nota == False
  end_part = df.iloc[:, -2:]

end_part_cols = [col for col in end_part.columns]
end_part[end_part_cols] = end_part[end_part_cols].replace('', 0)
end_part[end_part_cols] = end_part[end_part_cols].astype(int)

end_part = end_part.copy()

# End Part Organization
end_part_nota = end_part.iloc[:, [-2]]
end_part_nota = end_part_nota.copy()
# Calc of Rejected Votes
end_part_nota["Rejected Votes"] = 0
end_part_rej = end_part_nota.iloc[:, -1:]

# Dropping the extra Rejected Votes Column
end_part_nota.drop(end_part_nota.iloc[:, -1:], axis=1, inplace=True)


#----------------------------------------------------------------------------

# Assignment of Original Column Names

alpha_seg_1_S = ["S.No."]
alpha_seg_1 = ["P.S.No.", "Constituency"]
alpha_seg_2 = ["Male", "Female", "Others", "Total", "No. of Ten-dered Voters"]
parties = lst_of_parties 
mid_part_seg_1 = ["IND (TOTAL)"]
mid_part_seg_2 = ["Total Valid Votes"]
omega_seg_1= ["NOTA"]
omega_seg_2 = ["Rejected Votes"]

final_cols = alpha_seg_1_S + alpha_seg_1 + parties + mid_part_seg_1 + omega_seg_1 + mid_part_seg_2 + alpha_seg_2 + omega_seg_2 

# Concatenation for a Final Dataframe

df = pd.concat([start_part_seg_1_Sno, start_part_seg_1, mid_part, mid_part_ind_seg, end_part_nota, mid_part_val, start_part_seg_2, end_part_rej], axis=1)
df.columns = final_cols

#----------------------------------------------------------------------------

# Fixing the Constituency name

df["Constituency"] = consti_name

# -----------------------------------------------------------------------------

# Data Integrity Check Function

# checking the total valid votes and Total columns

def sum_integrity(df, val_votes, nota, total):
    flag = 0
    for index, row in df.iterrows():
        sum_value = row[val_votes] + row[nota]
        if sum_value == row[total]:
            print(f"Sum MATCHED for Row {index}, Data Integrity Valid")
        else:
            print(f"Sum DID NOT MATCH for Row {index}, Data Integrity Check FAILED")
            flag += 1
    flag_limit = 10
    if flag <= flag_limit:
        print(f"Sum Integrity Check Successful, the value of flags is {flag}")
    else:
        print(f"Sum Integrity Check Failed as value of Flags is {flag}")
    if flag <= flag_limit:
        return "Data Integrity Successful"
    else:
        return "Data Integrity Unsuccessful"

data_integrity_result = sum_integrity(df, "Total Valid Votes", "NOTA", "Total")

#-----------------------------------------------------------------------------

if data_integrity_result == "Data Integrity Successful":
    print("Form 20 READY FOR CONVERSION TO .XLSX")
    df.to_excel(f"{file_name}_processed.xlsx", index=False)
    print("FORM 20 Successfully Converted")
    print(df.shape)
else:
    print(f"Data Integrity Check was Unsuccessful as Flags are greater than the current flag limit, Advice Caution before Conversion")

df.head(15)


Sum MATCHED for Row 0, Data Integrity Valid
Sum MATCHED for Row 1, Data Integrity Valid
Sum MATCHED for Row 2, Data Integrity Valid
Sum MATCHED for Row 3, Data Integrity Valid
Sum MATCHED for Row 4, Data Integrity Valid
Sum MATCHED for Row 5, Data Integrity Valid
Sum MATCHED for Row 6, Data Integrity Valid
Sum MATCHED for Row 7, Data Integrity Valid
Sum MATCHED for Row 8, Data Integrity Valid
Sum MATCHED for Row 9, Data Integrity Valid
Sum MATCHED for Row 10, Data Integrity Valid
Sum MATCHED for Row 11, Data Integrity Valid
Sum MATCHED for Row 12, Data Integrity Valid
Sum MATCHED for Row 13, Data Integrity Valid
Sum MATCHED for Row 14, Data Integrity Valid
Sum MATCHED for Row 15, Data Integrity Valid
Sum MATCHED for Row 16, Data Integrity Valid
Sum MATCHED for Row 17, Data Integrity Valid
Sum MATCHED for Row 18, Data Integrity Valid
Sum MATCHED for Row 19, Data Integrity Valid
Sum MATCHED for Row 20, Data Integrity Valid
Sum MATCHED for Row 21, Data Integrity Valid
Sum MATCHED for Row 

,S.No.,P.S.No.,Constituency,SP,BJP,RSSP,SBSP,RBP,BSCP,RKP,PSP,JAP,IND ( Ashok Kumar ),IND ( Indrapal ),IND ( Parvati Nandan ),IND ( Satendra Kumar Panda ),IND ( Hari Om ),IND (TOTAL),NOTA,Total Valid Votes,Male,Female,Others,Total,No. of Ten-dered Voters,Rejected Votes
0,1,1,Etah,114,546,6,3,3,0,0,0,0,1,0,4,2,21,28,6,700,330,376,0,706,0,0
1,2,2,Etah,301,375,3,0,0,0,1,1,2,0,0,3,0,8,11,11,694,410,295,0,705,0,0
2,3,3,Etah,259,180,1,0,0,0,1,0,0,0,2,10,0,4,16,7,457,209,255,0,464,0,0
3,4,4,Etah,154,306,0,0,0,0,0,1,1,0,0,1,3,4,8,5,470,263,212,0,475,0,0
4,5,5,Etah,85,462,6,2,0,2,1,1,0,1,2,2,3,14,22,11,581,329,263,0,592,0,0
5,6,6,Etah,51,450,5,1,0,0,1,2,1,1,1,0,1,19,22,4,533,295,242,0,537,0,0
6,7,7,Etah,276,289,4,0,0,1,1,3,0,1,1,1,2,9,14,5,588,292,301,0,593,0,0
7,8,8,Etah,212,450,4,3,0,0,1,0,0,2,0,1,6,18,27,7,697,379,325,0,704,0,0
8,9,9,Etah,51,595,10,0,0,0,1,0,1,0,0,6,3,9,18,4,676,342,338,0,680,0,0
9,10,10,Etah,285,304,2,2,0,1,1,4,1,0,0,4,1,4,9,4,609,314,299,0,613,0,0
